In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split

from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Embedding, LSTM, Dense

from flask import Flask, render_template
import mysql.connector


In [2]:
import h5py
from tensorflow import keras
# H5 파일 경로
file_path = 'model\model.h5'

# H5 파일 열기
with h5py.File(file_path, 'r') as f:
    # MLP 모델 로드
    model = keras.models.load_model(f)

In [3]:

from flask import Flask # 플라스크 클래스 임포트
from flask import request, redirect

In [4]:
# 설문조사 데이터 전처리

def preprocessing(data):
  df = pd.DataFrame()
  df_one = pd.DataFrame()

  df['gender'] = data['gender'] # 성별
  df['aage06'] = data['age'] # 연령범주
  df['school0603'] = data['education'] # 학력
  df['grade0602'] = data['obstccentury'] # 장애정도
  df['ca063101'] = data['surveyselectone'] # 신체적인 능력 - 힘, 체력
  df['ca063102'] = data['surveyselecttwo'] # 신체적인 능력 - 이동, 빠르게 움직이기
  df['ca063103'] = data['surveyselectthree'] # 신체적인 능력 - 서있기, 불편한 자세 유지
  df['ca063104'] = data['surveyselectfour'] # 신체적인 능력 - 미세한 손동작
  df['ca063105'] = data['surveyselectfive'] # 컴퓨터를 활용하는 능력
  df['ca063106'] = data['surveyselectsix'] # 정보를 찾는 능력
  df['ca063107'] = data['surveyselectseven'] # 동료나 상사와 협력, 의사소통
  df['ca063108'] = data['surveyselecteight'] # 고객을 직접 또는 전화로 상대관리
  df['ca063110'] = data['surveyselectnine'] # 기계, 장비를 다루는 능력
  df['ca063111'] = data['surveyselectten'] # 수리, 계산하는 능력
  df['ca063112'] = data['surveyselecteleven'] # 외국어를 하는 능력
  df['ca063113'] = data['surveyselecttwelve'] # 규칙적이고 계획적인 시간관리 능력
  df['ca063114'] = data['surveyselectthirteen'] # 높은 집중력과 주의

  df_one['code'] = data['obstccode'] # 장애분류
  df_one['code'] = df_one['code'].astype(float)

  
  


#   1에서 15까지의 범위로 가정한 후, one-hot encoding을 수행합니다.
  value = data['obstccode']
  one_hot = np.zeros(15)
  one_hot[value-1] = 1

# 컬럼 이름 변경
  new_column_names = {f"code{i+1}": one_hot[i] for i in range(len(one_hot))}
  df_onehot = pd.DataFrame([new_column_names])
#   df_onehot = pd.DataFrame({'value': one_hot})
#   df_onehot.columns = [f"code{index+1}" for index in range(len(df_onehot.columns))]


 # df_encoded_data = pd.get_dummies(df_one['job_code']) # 장애분류 원핫인코딩
  
  # 직업코드 컬럼 이름 변경
#   new_column_names = {column: f"code{column}" for column in one_hot.columns}
#   one_hot = one_hot.rename(columns=new_column_names)


  # 장애코드와 df결합
  df = pd.concat([df, df_onehot], axis=1)

  return df

jobcode1 = {
    'jodcode1.0': [1.0],
    'jodcode2.0': [0],
    'jodcode3.0': [0],
    'jodcode4.0': [0],
    'jodcode5.0': [0],
    'jodcode6.0': [0],
    'jodcode7.0': [0],
    'jodcode8.0': [0],
    'jodcode9.0': [0]
}
jobcode2 = {
    'jodcode1.0': [0],
    'jodcode2.0': [1.0],
    'jodcode3.0': [0],
    'jodcode4.0': [0],
    'jodcode5.0': [0],
    'jodcode6.0': [0],
    'jodcode7.0': [0],
    'jodcode8.0': [0],
    'jodcode9.0': [0]
}
jobcode3 = {
    'jodcode1.0': [0],
    'jodcode2.0': [0],
    'jodcode3.0': [1.0],
    'jodcode4.0': [0],
    'jodcode5.0': [0],
    'jodcode6.0': [0],
    'jodcode7.0': [0],
    'jodcode8.0': [0],
    'jodcode9.0': [0]
}
jobcode4 = {
    'jodcode1.0': [0],
    'jodcode2.0': [0],
    'jodcode3.0': [0],
    'jodcode4.0': [1.0],
    'jodcode5.0': [0],
    'jodcode6.0': [0],
    'jodcode7.0': [0],
    'jodcode8.0': [0],
    'jodcode9.0': [0]
}
jobcode5 = {
    'jodcode1.0': [0],
    'jodcode2.0': [0],
    'jodcode3.0': [0],
    'jodcode4.0': [0],
    'jodcode5.0': [1.0],
    'jodcode6.0': [0],
    'jodcode7.0': [0],
    'jodcode8.0': [0],
    'jodcode9.0': [0]
}
jobcode6 = {
    'jodcode1.0': [0],
    'jodcode2.0': [0],
    'jodcode3.0': [0],
    'jodcode4.0': [0],
    'jodcode5.0': [0],
    'jodcode6.0': [1.0],
    'jodcode7.0': [0],
    'jodcode8.0': [0],
    'jodcode9.0': [0]
}
jobcode7 = {
    'jodcode1.0': [0],
    'jodcode2.0': [0],
    'jodcode3.0': [0],
    'jodcode4.0': [0],
    'jodcode5.0': [0],
    'jodcode6.0': [0],
    'jodcode7.0': [1.0],
    'jodcode8.0': [0],
    'jodcode9.0': [0]
}
jobcode8 = {
    'jodcode1.0': [0],
    'jodcode2.0': [0],
    'jodcode3.0': [0],
    'jodcode4.0': [0],
    'jodcode5.0': [0],
    'jodcode6.0': [0],
    'jodcode7.0': [0],
    'jodcode8.0': [1.0],
    'jodcode9.0': [0]
}
jobcode9 = {
    'jodcode1.0': [0],
    'jodcode2.0': [0],
    'jodcode3.0': [0],
    'jodcode4.0': [0],
    'jodcode5.0': [0],
    'jodcode6.0': [0],
    'jodcode7.0': [0],
    'jodcode8.0': [0],
    'jodcode9.0': [1.0]
}

jobcode1 = pd.DataFrame(jobcode1)
jobcode2 = pd.DataFrame(jobcode2)
jobcode3 = pd.DataFrame(jobcode3)
jobcode4 = pd.DataFrame(jobcode4)
jobcode5 = pd.DataFrame(jobcode5)
jobcode6 = pd.DataFrame(jobcode6)
jobcode7 = pd.DataFrame(jobcode7)
jobcode8 = pd.DataFrame(jobcode8)
jobcode9 = pd.DataFrame(jobcode9)


In [8]:
from flask import Flask
import mysql.connector

app = Flask(__name__) # 내장변수 name을 이용해 서버를 구동시키는 객체 생성
conn = mysql.connector.connect(
        host='project-db-cgi.smhrd.com',
        user='poyong',
        password='smhrd1234',
        database='poyong',
        port=3307  # MySQL 서버의 포트 번호
    )
@app.route("/predict", methods=["GET",'POST'])
def predict_jobcode() :
    
    # 쿼리스트링으로 넘어오는 userid 받아오기
    userid = request.args.get('userid')
    
# 쿼리 실행
    cursor = conn.cursor()
    query = "SELECT * FROM surveyresult WHERE userid = %s ORDER BY surveyresultidx DESC"
    cursor.execute(query, (userid,))
    data = cursor.fetchall()  # 튜플 형태로 결과를 가져옴 ([1,2,3], [9,5,51], ...), data[0]이 가장 최근
    print(data)
    

    # 커서 닫기
    cursor.close()

# 받아온 값을 데이터프레임으로 변환

    data = data[0]
    
    

    data= data[2:]
    
    
    df = pd.DataFrame([data], columns=['gender', 'age', 'education', 'obstccode', 'obstccentury', 'surveyselectone',
                                  'surveyselecttwo', 'surveyselectthree', 'surveyselectfour', 'surveyselectfive',
                                  'surveyselectsix', 'surveyselectseven', 'surveyselecteight', 'surveyselectnine',
                                  'surveyselectten', 'surveyselecteleven', 'surveyselecttwelve', 'surveyselectthirteen'])


    # 받아온 데이터 전처리                                             
    df = preprocessing(df)    
    # 전처리한 데이터와 미리 만들어둔 직업코드 데이터를 결합
    jobcodes = [jobcode1, jobcode2, jobcode3, jobcode4, jobcode5, jobcode6, jobcode7, jobcode8, jobcode9]
    df_jobcodes = []
    for i in range(1, 10):
        df_jobcode = pd.concat([df, jobcodes[i-1]], axis=1)
        globals()[f"df_jobcode{i}"] = df_jobcode # df_jobcode1, df_jobcode2, df_jobcode3, df_jobcode4,...
        df_jobcodes.append(df_jobcode) # 담을 필요가 있나 싶지만 일단 넣어둔다
       
    pd.set_option('display.max_columns', None)  # 모든 열 표시
    pd.set_option('display.expand_frame_repr', False)  # 줄 바꿈 없이 한 줄에 표시    
    # print(df_jobcodes[0])

    
    # # 추천할 직업을 담는 리스트
    # job_recommend = []   
    # # 추천도를 담는 리스트
    # #job_recommend_ddabong = []
    # #직업코드 수만큼 predict
    # for i in range(1, 10):
    #     df_jobcode = eval(f"df_jobcode{i}")
    #     globals()[f"pre{i}"] = model.predict(df_jobcode)
    #     print(globals()[f"pre{i}"])
    #     #job_recommend_ddabong.append(globals()[f"pre{i}"][0][2])
    #     # 만약 직업 추천도가 0.4 이상이라면 해당 직업을 추천한다.
    #     if globals()[f"pre{i}"][0][2] >= 0.4:
    #         job_recommend.append(i)
            
    import heapq
    top_recommendations = []
    for i in range(1, 10):
        df_jobcode = eval(f"df_jobcode{i}")
        globals()[f"pre{i}"] = model.predict(df_jobcode)
        # print(globals()[f"pre{i}"])
        recommendation_score = globals()[f"pre{i}"][0][2]
        heapq.heappush(top_recommendations, (recommendation_score, i))
        if len(top_recommendations) > 3:
            heapq.heappop(top_recommendations)

    job_recommend = [recommendation[1] for recommendation in top_recommendations]
    job_recommend.reverse()  # 가장 높은 값부터 출력되도록 순서를 뒤집습니다.
    print(job_recommend)


    # print("리스트의 길이",len(job_recommend))
    
    # for i in range(0, len(job_recommend)):
    #     print(f"{i+1}번째 리스트의 값은 {job_recommend[i]}")
    query_string = "&".join(f"predict{i+1}={job_recommend[i]}" for i in range(len(job_recommend)))

    redirect_url = f"http://localhost:8081/controller/surveyreuslt.do?{query_string}"

    return redirect(redirect_url)


if __name__ == "__main__" : # .py 파일에서 main함수 역할
    app.run(host="localhost", port="4500")

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:4500
Press CTRL+C to quit


[('gms', 70, 1, 5, 3, 4, 1, 2, 5, 4, 2, 3, 1, 4, 4, 5, 3, 1, 4, 4), ('gms', 69, 1, 5, 3, 4, 1, 5, 3, 4, 2, 4, 5, 2, 2, 4, 4, 3, 3, 3), ('gms', 68, 1, 4, 3, 4, 2, 5, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3), ('gms', 67, 1, 1, 1, 4, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5), ('gms', 66, 1, 1, 1, 4, 2, 5, 5, 5, 5, 5, 5, 5, 5, 2, 4, 2, 4, 2), ('gms', 65, 1, 1, 1, 4, 2, 2, 4, 4, 3, 3, 4, 5, 5, 2, 4, 2, 4, 2)]
1/1 [==============================] - 0s 20ms/step


127.0.0.1 - - [21/Jun/2023 18:05:36] "GET /predict?userid=gms HTTP/1.1" 302 -


[1, 2, 3]
